## Assignment3 - Retrieval Augmented Generation

In this asignment, you will develop a RAG solution to answer questions about a repository of research papers. The assignment requires you to parse the paper PDF files, chunk and index the data, and then design and execute an evaluation of the retriever results. In Naïve RAG, the query is compared to documents in the vector database for retrieval of the top N documents that match the query. The language model is then used to summarize the retrieved documents into an answer to the user query. Research papers are highly structured documents with technically deep content, in contrast to blogs, which contain more general and introductory content. This means that queries may be unlikely to match relevant chunks of the paper without additional processing, such as information extraction or summarization.

One approach to address this problem is to use the language model to generate answerable questions from chunks of each paper. The generated questions can then be indexed as "documents" in a vector database, and the user query can be matched against the most similar questions. By maintaining a mapping between the indexed, generated question and the paper chunk, the retrieval process can then produce the most relevant chunks for use in summarizing an answer to the user query.

## Setup the functions for prompting

In [5]:
from openai import OpenAI
client = OpenAI()

def prompt_model(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role": "user", 'content': prompt}
        ]
    )
    return completion.choices[0].message.content

## Parse data from source

In [10]:
import os, bibtexparser, pypdf, logging

# silence non-critical errors while parsing PDF files
logging.getLogger("pypdf").setLevel(logging.CRITICAL)

data_path = 'data/'
data = {}

files = os.listdir(data_path)
print('Reading %i files:' % len(files))
for f in files:
    path = os.path.join(data_path, f)

    # each datum will have at least these attributes
    d = {'filepath': None, 'title': None, 'text': None}

    # parse bibtex file, if exists
    if path.endswith('.bib'):
        if path[:-4] in data:
            d = data[path[:-4]]

        bib = bibtexparser.load(open(path, 'r'))
        if 'title' in bib.entries[0]:
            d['title'] = bib.entries[0]['title']
            data[path[:-4]] = d

    # parse pdf text, if exists
    if path.endswith('.pdf'):
        if path[:-4] in data:
            d = data[path[:-4]]

        print('  File: %s' % f)
        text = ''
        reader = pypdf.PdfReader(path)
        for page in reader.pages:
            text += page.extract_text()
        d['filepath'] = path
        d['text'] = text
        data[path[:-4]] = d

data = [d for d in data.values()]

Reading 53 files:
  File: 2023.findings-emnlp.620.pdf
  File: 29728-Article Text-33782-1-2-20240324-3.pdf
  File: 2024.acl-long.642.pdf
  File: 2021.findings-emnlp.320.pdf
  File: 2020.coling-main.207.pdf
  File: 2202.01110v2.pdf
  File: 2212.14024v2.pdf
  File: 2024.emnlp-industry.66.pdf
  File: 8917_Retrieval_meets_Long_Cont.pdf
  File: NeurIPS-2023-lift-yourself-up-retrieval-augmented-text-generation-with-self-memory.pdf
  File: NeurIPS-2023-leandojo-theorem-proving-with-retrieval-augmented-language-models.pdf
  File: NeurIPS-2020-retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf
  File: 2023.acl-long.557.pdf
  File: tacl_a_00605.pdf
  File: 3637870.pdf
  File: 2023.emnlp-main.495.pdf
  File: 3626772.3657834.pdf
  File: 2402.19473v6.pdf
  File: 3626772.3657957.pdf
  File: 2024.eacl-demo.16.pdf
  File: 967_generate_rather_than_retrieve_.pdf
  File: 23-0037.pdf
  File: 2022.naacl-main.191.pdf
  File: 2312.10997v5.pdf
  File: 947_Augmented_Language_Models_.pdf


## Pre-process the data

Prior to indexing and chunking the data, the data may need to be pre-processed. This can be done to remove portions of the data irrelevant to queries to reduce mismatches between the user query and the index. This is not required for this assignment.

## Chunk data and generate indices

User queries will be matched to indexes that best approximate the text chunks used to summarize an answer. For this assignment, you may chunk the text and then prompt the model to generate questions that are answerable by the text. The generated questions can then be used as the "documents" stored in the vector database.

## Build the vector database

When building the vector database, be sure to maintain a mapping between the generated questions and the chunks that can be used later to retrieve the chunks from the most similar indices to the user query provided.

You may also add the function to query the vector database that you will use later.

In [11]:
import chromadb



## Conduct experiments to evaluate user queries

Report your average precision, recall and F1 score. You are welcome to sample the model multiple times for each query when computing your average, or you may sample once per query.

In [ ]:
from bert_score import BERTScorer

scorer = BERTScorer(model_type='bert-base-uncased')